# LIAR DETECTION GROUP PROJECT

Run the cell below to import packages.

In [87]:
from __future__ import absolute_import
from __future__ import print_function
from __future__ import division

import json, os, re, shutil, sys, time
from importlib import reload
import collections, itertools
from functools import reduce

#import unittest
from IPython.display import display, HTML
#from sklearn.utils import shuffle
# NLTK for NLP utils and corpora
#import nltk
from collections import defaultdict
# NumPy and TensorFlow
import numpy as np
import pandas as pd
#import tensorflow as tf
import pickle
import dill
# Helper libraries
from w266_common import utils, vocabulary, tf_embed_viz
import timeit  #For timing
from sklearn.feature_extraction.text import CountVectorizer
#from project_files import pdio # for saving and loading dataframes

### Load data
Loading data from Pre-Processing step Part 1

In [158]:
#Read pkl file from part 1
# all_data = pd.read_pickle('parsed_data/df_alldata1.pkl')
all_data = pd.read_pickle('parsed_data/df_liarpolitifact_data_embed.pkl')
all_data.info(memory_usage='deep', verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23258 entries, 0 to 23257
Data columns (total 6 columns):
target            23258 non-null int64
title             23258 non-null object
title_tokcan      23258 non-null object
title_POS         23258 non-null object
binary_target     23258 non-null int64
embedded_title    23258 non-null object
dtypes: int64(2), object(4)
memory usage: 19.1 MB


In [159]:
all_data[:5]

,target,title,title_tokcan,title_POS,binary_target,embedded_title
0,1,Says 31 percent of Texas physicians accept all...,"[says, <number>, percent, of, texas, physician...","[V, $, N, P, ^, N, V, D, A, ^, N, ,, R, P, $, ...",1,"[[0.11797, 0.21126, 0.29075, -0.021211, 0.7819..."
1,2,''Both Democrats and Republicans are advocatin...,"['', both, democrats, and, republicans, are, a...","[,, D, N, &, N, V, V, P, D, N, P, N, N, V, P, ...",-1,"[[0.0028594, 0.19457, -0.19449, -0.037583, 0.9..."
2,0,A Republican-led softening of firearms trainin...,"[a, republican-led, softening, of, firearms, t...","[D, A, N, P, N, N, N, V, D, A, N, V, V, V, P, ...",1,"[[0.21705, 0.46515, -0.46757, 0.10082, 1.0135,..."
3,5,The first tweet was sent from Austin.,"[the, first, tweet, was, sent, from, austin, .]","[D, A, N, V, V, P, ^, ,]",0,"[[0.418, 0.24968, -0.41242, 0.1217, 0.34527, -..."
4,2,Georgia has the countrys second highest number...,"[georgia, has, the, countrys, second, highest,...","[^, V, D, N, A, A, N, P, A, N, N, N, ,]",-1,"[[-1.3427, 0.4592, 0.19281, 0.71305, -0.5934, ..."


# Char pre-process
Make it easier to select different sources

In [106]:
#select column to train on 
# mini_data = all_data[:10]

tok_char_n = lambda a, ngram_len: [a[i:i+ngram_len] for i in range(len(a))][:-ngram_len]

n_gram_len = 2
field_name='title'
new_field_name = field_name+'_pad'
tok_field_name = field_name+'_tok'
tok_int_field_name = field_name+'_tok_int'

# find 95% quantile
text_lens=[len(i) for i in all_data[field_name]]
pad_len = int(np.percentile(text_lens, 95))

# truncate/pad new field to 95th percentile
all_data[new_field_name] = [(" ".join(i.lower().split()) + pad_len * ' ')[:pad_len] for i in all_data[field_name]]

# tokenize
all_data[tok_field_name] = [tok_char_n(i,n_gram_len) for i in all_data[new_field_name]]

# Define helper functions
Helper functions for processing

In [99]:
# pulled from a StackOverflow post

def train_validate_test_split(df, train_percent=.6, validate_percent=.2, seed=None):
    np.random.seed(seed)
    perm = np.random.permutation(df.index)
    m = len(df.index)
    train_end = int(train_percent * m)
    validate_end = int(validate_percent * m) + train_end
    train = df.ix[perm[:train_end]]
    validate = df.ix[perm[train_end:validate_end]]
    test = df.ix[perm[validate_end:]]
    return train, validate, test

0    [br, ra, ai, in, ni, ia, ac, c ,  g, ge, et, t...
1    [wi, in, nd, do, ow, ws, s ,  1, 10, 0 ,  i, i...
2    [st, tu, un, nn, ni, in, ng, g ,  s, st, to, o...
3    [no, or, rt, th, h ,  k, ko, or, re, ea, a', '...
4    [wh, hi, it, te, e ,  h, ho, ou, us, se, e ,  ...
Name: title_tok, dtype: object

# Process character n-grams
Create character n-grams for each of the news entries. This gives me the vocabulary.

In [133]:
vec = []
ngram = []
start_ngram = 2
vocab = []
for ngram_len in range(start_ngram,2):
    i = ngram_len-2
    vec.append(CountVectorizer(analyzer='char', binary=True, ngram_range=(n_gram_len, n_gram_len)))
    ngram.append(vec[i].fit_transform(all_data[field_name])
    vocab.append(vec[i].vocabulary_)
    vocab[i]['  '] = len(vocab)
    vocab[i]['oov'] = len(vocab)

    tok = lambda voc, toks: [voc[i].get(t,default) for t in toks]

    all_data[tok_int_field_name] = [tok(vocab[i], j) for j in all_data[tok_field_name]]

    all_data[tok_int_field_name][:10]

0    [793, 1441, 740, 1099, 1295, 1086, 734, 804, 3...
1    [1660, 1099, 1290, 875, 1358, 1668, 1473, 12, ...
2    [1512, 1559, 1595, 1300, 1295, 1099, 1293, 984...
3    [1301, 1353, 1460, 1546, 1032, 36, 1168, 1353,...
4    [1659, 1056, 1105, 1543, 893, 33, 1061, 1356, ...
5    [1209, 1347, 1183, 64, 32, 1007, 922, 1353, 14...
6    [1056, 1097, 1206, 1195, 749, 1465, 1709, 28, ...
7    [1445, 923, 1399, 1583, 787, 1203, 1088, 817, ...
8    [1159, 919, 1206, 1218, 1725, 745, 1300, 1291,...
9    [1770, 1098, 1243, 777, 733, 798, 1656, 896, 1...
Name: title_tok_int, dtype: object

# Build and run Keras LSTM


In [155]:
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.layers import Embedding
from keras.layers import LSTM

all_data_train, all_data_dev, all_data_test = train_validate_test_split(all_data, train_percent = .7, validate_percent=.15)

x_train = np.array(list(all_data_train[tok_int_field_name]))
print("x_train: ", x_train.shape)
y_train = np.reshape(list(all_data_train['binary_target']),[-1,1])
print("y_train: ", y_train.shape)
x_test = np.array(list(all_data_dev[tok_int_field_name]))
print("x_test: ", x_test.shape)
y_test = np.reshape(list(all_data_dev['binart_target']),[-1,1])
print("y_test: ", y_test.shape)

max_features = len(vocab)

model = Sequential()
model.add(Embedding(max_features, output_dim=256))
model.add(LSTM(128))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

model.fit(x_train, y_train, batch_size=16, epochs=5)
score = model.evaluate(list(x_test), list(y_test), batch_size=16)


# define model
# model = Sequential()
# model.add(LSTM(75, input_shape=(X.shape[1], X.shape[2])))
# model.add(Dense(vocab_size, activation='softmax'))
print(model.summary())

/opt/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:7: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  import sys
/opt/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:8: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  
/opt/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:9: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  if __name__ == '__main__':


x_train:  (31428, 131)
y_train:  (31428, 1)
x_test:  (6734, 131)
y_test:  (6734, 1)
Epoch 1/2
31428/31428 [==============================] - 334s 11ms/step - loss: 0.2326 - acc: 0.9011
Epoch 2/2
31428/31428 [==============================] - 331s 11ms/step - loss: 0.1178 - acc: 0.9626


ValueError: Error when checking model input: the list of Numpy arrays that you are passing to your model is not the size the model expected. Expected to see 1 array(s), but instead got the following list of 6734 arrays: [array([[1659],
       [1056],
       [1097],
       [1199],
       [ 893],
       [  45],
       [1546],
       [1052],
       [ 893],
       [  48],
       [1665],
       [1353],
       [1452],
    ...

In [147]:
list(all_data_dev[tok_int_field_name][:10])

[[1921,
  1351,
  1385,
  921,
  1270,
  38,
  1250,
  1099,
  1290,
  865,
  911,
  890,
  1935,
  37,
  1203,
  1087,
  781,
  925,
  1441,
  743,
  1213,
  1473,
  27,
  777,
  745,
  1270,
  45,
  1556,
  1461,
  1594,
  1256,
  1374,
  312,
  1513,
  1597,
  1395,
  1394,
  1353,
  1460,
  1547,
  1099,
  1293,
  984,
  26,
  747,
  1381,
  749,
  1460,
  1551,
  1246,
  921,
  1306,
  1524,
  33,
  1067,
  1595,
  1306,
  1543,
  925,
  1459,
  1473,
  2015,
  2015,
  2015,
  2015,
  2015,
  2015,
  2015,
  2015,
  2015,
  2015,
  2015,
  2015,
  2015,
  2015,
  2015,
  2015,
  2015,
  2015,
  2015,
  2015,
  2015,
  2015,
  2015,
  2015,
  2015,
  2015,
  2015,
  2015,
  2015,
  2015,
  2015,
  2015,
  2015,
  2015,
  2015,
  2015,
  2015,
  2015,
  2015,
  2015,
  2015,
  2015,
  2015,
  2015,
  2015,
  2015,
  2015,
  2015,
  2015,
  2015,
  2015,
  2015,
  2015,
  2015,
  2015,
  2015,
  2015,
  2015,
  2015,
  2015,
  2015,
  2015,
  2015,
  2015,
  2015,
  2015,
  2015,
  2

In [156]:
# x_train = np.array(list(all_data_train[tok_int_field_name]))
# print("x_train: ", x_train.shape)
# y_train = np.array(list(all_data_train['target']))
# print("y_train: ", y_train.shape)
# x_test = np.array(list(all_data_dev[tok_int_field_name]))
# print("x_test: ", x_test.shape)
# y_test = np.reshape(list(all_data_dev['target']),[1,-1])
# print("y_test: ", y_test.shape)

score = model.evaluate(x_test, y_test, batch_size=16)

6734/6734 [==============================] - 15s 2ms/step


In [157]:
score

[0.10409376217936916, 0.9642114642291668]